# Solution to ecoSPEC-challenge Task 2

Due to limited compute resources, I will be using Google Colab to run inferences. The following code is from my main.py, but I am running the mistralai/Mistral-Nemo-Instruct-2407 model through the Hugging Face Inference API instead.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/https://github.com/Malikbadmus/ecoSPEC-challenge-2025/blob/main/tablegen/Tablegen.ipynb)


In [1]:
import torch

if torch.cuda.is_available():
    print("✅ GPU is available!")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("❌ No GPU available. Go to Runtime > Change runtime type and select GPU.")

✅ GPU is available!
GPU Name: Tesla T4


In [73]:
!pip install python-docx
!pip install docx2pdf
!pip install requests

In [75]:
import os
import json
import re
from typing import List, Optional

import torch
import requests
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [ ]:
class TableFiller:
    def __init__(self, model_name: str = "mistralai/Mistral-Nemo-Instruct-2407", api_key: str = ""):
        print(f"Using Hugging Face API model: {model_name}")
        self.api_key = api_key
        self.model_name = model_name
        self.url = f"https://api-inference.huggingface.co/models/{model_name}"
        self.headers = {"Authorization": f"Bearer {api_key}"}

    def format_prompt(self, user_prompt: str, row_headers: Optional[List[str]] = None,
                      column_headers: Optional[List[str]] = None, num_rows: int = 5, num_cols: int = 3) -> str:
        prompt = f"Generate table content for: {user_prompt}\n"
        if row_headers:
            prompt += f"Row headers: {', '.join(row_headers)}\n"
        if column_headers:
            prompt += f"Column headers: {', '.join(column_headers)}\n"
        if not row_headers and not column_headers:
            prompt += f"Generate a {num_rows}x{num_cols} table with appropriate values.\n"
        prompt += "\nReturn ONLY a JSON array of arrays."
        return prompt

    def call_hf_api(self, prompt: str) -> list:
        payload = {"inputs": prompt}
        try:
            response = requests.post(self.url, headers=self.headers, json=payload)
            response.raise_for_status()
            output = response.json()

            text = output[0]["generated_text"]
            print("Raw API Output:\n", text)

            match = re.search(r'(\[\s*\[.*?\]\s*\])', text, re.DOTALL)
            if match:
                json_str = match.group(1)
                parsed = json.loads(json_str)
                return parsed
            else:
                print("Could not extract valid JSON array.")
                return []
        except requests.exceptions.RequestException as e:
            print("Request failed:", e)
        except Exception as e:
            print("Unexpected error:", e)
        return []

    def generate_table(self, prompt: str, row_headers: Optional[List[str]] = None,
                   column_headers: Optional[List[str]] = None,
                   num_rows: int = 5, num_cols: int = 3) -> List[List[str]]:


        full_prompt = f"Generate a table with the following rows: {', '.join(row_headers)} and columns: {', '.join(column_headers)}. Return only the JSON array of arrays. No comments, no explanation."

        print("Prompt sent to API:\n", full_prompt)

        result = self.call_hf_api(full_prompt)
        print("Raw API Output:\n", result)

        return (result)


    def save_to_docx(self, table_data: List[List[str]], file_path: str,
                 intro_text: Optional[str] = None, title: Optional[str] = None):
        doc = Document()
        if title:
            heading = doc.add_heading(title, level=1)
            heading.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        if intro_text:
            doc.add_paragraph(intro_text)


        max_cols = max(len(row) for row in table_data)
        rows = len(table_data)

        table = doc.add_table(rows=rows, cols=max_cols)
        table.style = 'Table Grid'

        for i, row in enumerate(table_data):
            for j in range(max_cols):
                value = str(row[j]) if j < len(row) else ""
                table.cell(i, j).text = value

        doc.save(file_path)
        print(f"Saved table to {file_path}")



In [ ]:
api_key = ""
model = "mistralai/Mistral-Nemo-Instruct-2407"

table_filler = TableFiller(model_name=model, api_key=api_key)

prompt = "Nutritional information of common fruits"
row_headers = ["Apple", "Banana", "Orange", "Grapes", "Strawberry"]
column_headers = ["Calories", "Sugar (g)", "Vitamin C (mg)"]

table_data = table_filler.generate_table(
    prompt,
    row_headers=row_headers,
    column_headers=column_headers
)

table_filler.save_to_docx(
    table_data,
    file_path="fruit_nutrition.docx",
    intro_text="This table contains nutritional information of common fruits.",
    title="Fruit Nutrition Table"
)


Using Hugging Face API model: mistralai/Mistral-Nemo-Instruct-2407
Prompt sent to API:
 Generate a table with the following rows: Apple, Banana, Orange, Grapes, Strawberry and columns: Calories, Sugar (g), Vitamin C (mg). Return only the JSON array of arrays. No comments, no explanation.
Raw API Output:
 Generate a table with the following rows: Apple, Banana, Orange, Grapes, Strawberry and columns: Calories, Sugar (g), Vitamin C (mg). Return only the JSON array of arrays. No comments, no explanation. [
 ["Calories", "Sugar (g)", "Vitamin C (mg)"],
 ["Apple", 52, 4.2, 4.6],
 ["Banana", 89, 12.2, 8.7],
 ["Orange", 39, 7.0, 53.2],
 ["Grapes", 104, 16.25, 1.2],
 ["Strawberry", 49, 4.89, 58.8]
]
Raw API Output:
 [['Calories', 'Sugar (g)', 'Vitamin C (mg)'], ['Apple', 52, 4.2, 4.6], ['Banana', 89, 12.2, 8.7], ['Orange', 39, 7.0, 53.2], ['Grapes', 104, 16.25, 1.2], ['Strawberry', 49, 4.89, 58.8]]
Saved table to fruit_nutrition.docx
